In [1]:
from pynq import Overlay
from pynq.lib.video import *

overlay = Overlay("/home/xilinx/parameter/base.bit")

In [2]:
imgen_vdma = overlay.video.axi_vdma_0.readchannel

In [3]:
pixgen = overlay.pixel_generator_0
pixgen.register_map

RegisterMap {
  gp0 = Register(value=0),
  gp1 = Register(value=0),
  gp2 = Register(value=0),
  gp3 = Register(value=0),
  gp4 = Register(value=0),
  gp5 = Register(value=0),
  gp6 = Register(value=0),
  gp7 = Register(value=0)
}

In [4]:
#fixed parameters
w = 1024
h = 768
v_w = 640
v_h = 480
speed = 5
x_bound = 120 
y_bound = 90

In [5]:
def dec_to_hex(number0, integers=3, fractional=28):
    # Total number of bits (1 sign bit + integer bits + fractional bits)
    number = number0
    total_bits = 1 + integers + fractional

    if number < 0:
        is_negative = True
        number = -number
    else:
        is_negative = False

    integer_part = int(number)
    fractional_part = number - integer_part

    # Convert integer part to binary with the specified number of bits
    integer_bin = bin(integer_part)[2:].zfill(integers)

    # Convert fractional part to binary with the specified number of bits
    fractional_bin = ""
    for _ in range(fractional):
        fractional_part *= 2
        bit = int(fractional_part)
        fractional_bin += str(bit)
        fractional_part -= bit

    combined_bin = f"{integer_bin[:integers]}{fractional_bin[:fractional]}"
    
    if is_negative:
        inverted_bin = ''.join('1' if bit == '0' else '0' for bit in combined_bin)
        twos_complement_bin = bin(int(inverted_bin, 2) + 1)[2:].zfill(integers + fractional)
    else:
        twos_complement_bin = combined_bin

    final_bin = f"{'1' if is_negative else '0'}{twos_complement_bin}"


    hex_representation = hex(int(final_bin, 2))[2:].zfill((total_bits + 3) // 4)
    
    # Return an int, from the HEX representation
    return int(hex_representation, 16)

In [6]:
def get_zoom_factor(w, r1, r2):
    #We assume that we always have a 4/3 ratio
    if r2 > r1:
        d = r2-r1
    else:
        d = r1-r2
    z_f = d/w
    return z_f

In [7]:
def to_32bit_hex(w, h):
    if not (0 <= w < 2**16):
        raise ValueError("w must be a 16-bit number (0 <= w < 65536)")
    if not (0 <= h < 2**16):
        raise ValueError("h must be a 16-bit number (0 <= h < 65536)")
    
    combined = (h << 16) | w
    
    return combined

In [43]:
def update_reg_map(colour, zoom, Im, Re, dim, it):
    pixgen.register_map.gp0 = colour #colour
    print(colour)
    pixgen.register_map.gp1 = zoom   #Zoom
    pixgen.register_map.gp2 = Im     #Im_upper
    pixgen.register_map.gp3 = Re     #Re_lower
    pixgen.register_map.gp4 = dim    #dimensions 
    pixgen.register_map.gp5 = it     #max_iterations

In [9]:
import cv2
import numpy as np
from pynq.buffer import PynqBuffer
from pynq import allocate

def resize_small_frame(frame: PynqBuffer, new_height: int, new_width: int) -> PynqBuffer:
    # Convert the PynqBuffer to a NumPy array
    frame_np = np.array(frame)
    
    # Resize the array using OpenCV
    resized_np = cv2.resize(frame_np, (new_width, new_height))
    
    # Allocate a new PynqBuffer for the resized array
    resized = allocate(shape=resized_np.shape, dtype=resized_np.dtype)
    
    # Copy the resized array into the new PynqBuffer
    np.copyto(resized, resized_np)
    
    return resized

In [10]:
import numpy as np
from pynq.buffer import PynqBuffer
from pynq import allocate

def crop_big_frame(big_frame: PynqBuffer, x: int, y: int, zoom_x:int = 0, zoom_y:int = 0) -> PynqBuffer:
    # Convert the PynqBuffer to a NumPy array
     
    crop_height = 480
    crop_width = 640
    zoom_x = int(zoom_x //2)
    zoom_y = int(zoom_y //2)
    big_frame_np = np.array(big_frame)
    
    # Ensure the crop area is within the bounds
    max_y, max_x, _ = big_frame_np.shape
    if y + crop_height > max_y:
        y = max_y - crop_height
    if x + crop_width > max_x:
        x = max_x - crop_width
    if y < 0:
        y = 0
    if x < 0:
        x = 0
    
    # Crop the big np.array 
    small_frame_np = big_frame_np[y-zoom_y:y + crop_height+zoom_y, x-zoom_x:x + crop_width+ zoom_x]
    #print(len(small_frame_np),zoom_y,zoom_x)
    # Allocate a new PynqBuffer for the np.array
    small_frame = allocate(shape=small_frame_np.shape, dtype=small_frame_np.dtype)
    
    # Copy the np.array into PynqBuffer
    np.copyto(small_frame, small_frame_np)
    
    # In case of zoom - resize to the correct format for HDMI display
    small_frame = resize_small_frame(small_frame, 480, 640) 
    
    return small_frame

In [11]:
videoMode = common.VideoMode(w,h, 24)
imgen_vdma.mode = videoMode
imgen_vdma.start()

In [12]:
screenMode = common.VideoMode(v_w, v_h, 24)

In [13]:
hdmi_out = overlay.video.hdmi_out
hdmi_out._vdma = overlay.video.axi_vdma #Use the correct VDMA!
hdmi_out.configure(screenMode)
hdmi_out.start()

In [14]:
#reset to orignial values
Re = -2
colour = 2
r2 = 2
zoom = get_zoom_factor(w, Re, r2)
Im = 1.5
dim = to_32bit_hex(w, h)
it = 0x3ff
x = (w-v_w)//2
y = (h-v_h)//2

center_x = (w-v_w)//2
center_y = (h-v_h)//2

image_w = 640
image_h = 480
update_reg_map(colour, dec_to_hex(zoom),  dec_to_hex(Im),  dec_to_hex(Re), dim, it)
loading = False

In [ ]:
def reset():
    #reset to orignial values
    Re = -2
    colour = 2
    r2 = 2
    zoom = get_zoom_factor(w, Re, r2)
    Im = 1.5
    dim = to_32bit_hex(w, h)
    it = 0x3ff
    x = (w-v_w)//2
    y = (h-v_h)//2

    center_x = (w-v_w)//2
    center_y = (h-v_h)//2

    image_w = 640
    image_h = 480
    update_reg_map(colour, dec_to_hex(zoom),  dec_to_hex(Im),  dec_to_hex(Re), dim, it)
    loading = True

In [15]:
frame = imgen_vdma.readframe()

In [16]:
def get_image_parameters(zoom, Re, Im, image_w, image_h):
    # Distances 
    d_x = zoom*w
    d_y = zoom*h
    
    small_d_x = (image_w*d_x)/w
    small_d_y = (image_h*d_y)/h
    
    
    # Proportion parameters
    p_x = (((v_w-image_w)/2)+x)/w
    p_y = (((v_h-image_h)/2)+y)/h
    print("p", p_x, p_y)
    
    
    image_Re = Re + (d_x*p_x)
    image_Im = Im - (d_y*p_y)
    

    
    print("Small_d", small_d_x, small_d_y)
    return image_Re, image_Im, small_d_x , small_d_y

In [17]:
def generate_new_frame(zoom, Re, Im, image_w, image_h):
    image_Re, image_Im, d_x, d_y = get_image_parameters(zoom, Re, Im, image_w, image_h)
    
    big_d_x = (w*d_x)/v_w
    big_d_y = (h*d_y)/v_h
        
    c_x = d_x/2 + min(-image_Re, image_Re)
    c_y =  max(-image_Im, image_Im) - d_y/2 
    print("centre", c_x, c_y)
    
    new_Re = c_x - big_d_x/2
    new_Im = c_y + big_d_y/2
    
    print("big_distance", big_d_x, big_d_y)
    
    new_zoom = get_zoom_factor(w, new_Re, new_Re+big_d_x) 
    update_reg_map(colour, dec_to_hex(new_zoom), dec_to_hex(new_Im), dec_to_hex(new_Re), dim, it)
    
    return new_Re, new_Im, new_zoom, image_Re, image_Im

In [18]:
import time
start_time = None
loading = False
loading_time = 1.2

In [19]:
# movemement
def move_right (frame, x, y):
    x += speed
    out = False
    if image_w+x > w - x_bound:
        out = True
    cropped_frame = crop_big_frame(frame, x, y)
    return cropped_frame, x , y, out

def move_left (frame, x, y):
    x -= speed
    out = False
    if x_bound > x-(image_w-v_w)/2:
        out = True
    cropped_frame = crop_big_frame(frame, x, y)
    return cropped_frame, x , y, out

def move_down (frame, x, y):
    y += speed
    out = False
    if image_h+y > h - y_bound:
        out = True
    cropped_frame = crop_big_frame(frame, x, y)
    return cropped_frame, x ,y, out

def move_up (frame, x, y):
    y -= speed
    out = False
    if y_bound > y - (image_h - v_h)/2:
        out = True
    cropped_frame = crop_big_frame(frame, x, y)
    return cropped_frame, x, y, out
def zoom_im(frame, x, y):
    print("zooom")
    
def zoom_out(frame, x, y, image_w, image_h):
    out = False
    image_w += 8
    image_h += 6
    #print(image_w , w-x_bound)
    #print(image_h , h-y_bound)
    if image_w > w-(2*x_bound) or image_h > h-(2*y_bound):
        out = True
    #cropped_frame = zoom_out_pynq_buffer(frame, 1.1)
    z_x = image_w - 640
    z_y = image_h - 480
    cropped_frame = crop_big_frame(frame, x, y, z_x, z_y)
    return cropped_frame, image_w, image_h, out
def zoom_in(frame, x, y, image_w, image_h):
    out = False
    image_w -= 8
    image_h -= 6
    #print(image_w , w-x_bound)
    #print(image_h , h-y_bound)
    if image_w < v_w-x_bound or image_h < v_h-y_bound:
        out = True
    #cropped_frame = zoom_out_pynq_buffer(frame, 1.1)
    z_x = image_w - 640
    z_y = image_h - 480
    cropped_frame = crop_big_frame(frame, x, y, z_x, z_y)
    return cropped_frame, image_w, image_h, out

In [27]:
# Make 100 movements to one side
for i in range(100):
    #cropped_frame, x, y, out = move_left(frame, x, y) # change to choose movement
    cropped_frame, image_w, image_h, out= zoom_out(frame, x, y, image_w, image_h) # change to choose zoom
    print(x, y, image_w, image_h)
        #print(x, y, image_w, image_h)

    if out and not loading:
        print("out")
        new_Re, new_Im, zoom, image_Re, image_Im = generate_new_frame(zoom, Re, Im, image_w, image_h)
        og_x, og_y = x, y
        og_w, og_h = image_w, image_h
        print("Image", image_Re, image_Im)
        print("Frame", new_Re, new_Im)
        #new_x, new_y = get_new_xy(x, y, Re, new_Re, Im, new_Im, image_Re, image_Im)
        #print(new_x, new_y)
        Re = new_Re
        Im = new_Im

        loading = True
        start_time = time.time()
        x_buffer = 0
        y_buffer = 0

    elif loading:
        if time.time() - start_time >= loading_time:
            bg_frame = imgen_vdma.readframe()
            frame = bg_frame
            image_w = 640 + (image_w - og_w)
            image_h = 480 + (image_h - og_h)
            loading = False
            x, y = center_x + (x - og_x), center_y + (y - og_y)
            print("image loaded")
        else:
            print("buffer")

    hdmi_out.writeframe(cropped_frame)

192 144 648 486
192 144 656 492
192 144 664 498
192 144 672 504
192 144 680 510
192 144 688 516
192 144 696 522
192 144 704 528
192 144 712 534
192 144 720 540
192 144 728 546
192 144 736 552
192 144 744 558
192 144 752 564
192 144 760 570
192 144 768 576
192 144 776 582
192 144 784 588
192 144 792 594
out
p 0.11328125 0.11328125
Small_d 3.09375 2.3203125
centre 0.0 0.0
big_distance 4.95 3.7125
Image -1.546875 1.16015625
Frame -2.475 1.85625
192 144 800 600
buffer
192 144 808 606
buffer
192 144 816 612
buffer
192 144 824 618
buffer
192 144 832 624
buffer
192 144 840 630
buffer
192 144 848 636
buffer
192 144 856 642
buffer
192 144 864 648
buffer
192 144 872 654
image loaded
192 144 728 546
192 144 736 552
192 144 744 558
192 144 752 564
192 144 760 570
192 144 768 576
192 144 776 582
192 144 784 588
192 144 792 594
out
p 0.11328125 0.11328125
Small_d 3.828515625 2.8713867187500006
centre 0.0 -4.440892098500626e-16
big_distance 6.125625 4.5942187500000005
Image -1.9142578125 1.4356933593

In [37]:
def handle_state(out, loading):
    if out and not loading:
        print("out")
        new_Re, new_Im, zoom, image_Re, image_Im = generate_new_frame(zoom, Re, Im, image_w, image_h)
        og_x, og_y = x, y
        og_w, og_h = image_w, image_h
        print("Image", image_Re, image_Im)
        print("Frame", new_Re, new_Im)
        #new_x, new_y = get_new_xy(x, y, Re, new_Re, Im, new_Im, image_Re, image_Im)
        #print(new_x, new_y)
        Re = new_Re
        Im = new_Im

        loading = True
        start_time = time.time()
        x_buffer = 0
        y_buffer = 0

    elif loading:
        if time.time() - start_time >= loading_time:
            bg_frame = imgen_vdma.readframe()
            frame = bg_frame
            image_w = 640 + (image_w - og_w)
            image_h = 480 + (image_h - og_h)
            loading = False
            x, y = center_x + (x - og_x), center_y + (y - og_y)
            print("image loaded")
        else:
            print("buffer")
    hdmi_out.writeframe(cropped_frame)
    return loading, start_time

In [39]:
def handle_movement(message, loading, start_time):
    if message == "move_left_down":
        cropped_frame, x, y, out = move_left(frame, x, y)
        cropped_frame, x, y, out = move_down(frame, x, y)
    if message == "move_left_up":
        cropped_frame, x, y, out = move_left(frame, x, y)
        cropped_frame, x, y, out = move_up(frame, x, y)
    if message == "move_right_down":
        cropped_frame, x, y, out = move_right(frame, x, y)
        cropped_frame, x, y, out = move_down(frame, x, y)
    if message == "move_right_up":
        cropped_frame, x, y, out = move_right(frame, x, y)
        cropped_frame, x, y, out = move_up(frame, x, y)
    if message == "move_left":
        cropped_frame, x, y, out = move_left(frame, x, y)
    if message == "move_right":
        cropped_frame, x, y, out = move_right(frame, x, y)
    if message == "move_up":
        cropped_frame, x, y, out = move_up(frame, x, y)
    if message == "move_down":
        cropped_frame, x, y, out = move_down(frame, x, y)
    if message == "zoom_in":
        cropped_frame, image_w, image_h, out= zoom_in(frame, x, y, image_w, image_h)
    if message == "zoom_out":
        cropped_frame, image_w, image_h, out= zoom_in(frame, x, y, image_w, image_h)
    loading, start_time = handle_state(out, loading)
    return loading, start_time

presets = {
    1 : (get_zoom_factor(w, -0.9, -0.6), -0.9, 0.2),
    2 : (get_zoom_factor(w, -0.8, -0.6), -0.9, 0.06)
    
}

In [49]:
import socket
host='0.0.0.0'
port=12337

In [ ]:
# Create a TCP/IP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the address and port
server_socket.bind((host, port))

mapping = {
    1 : "move_left_down",
    2 : "move_left_up",
    3 : "move_right_down",
    4 : "move_right_up",
    5 : "move_left",
    6 : "move_right",
    7 : "move_up",
    8 : "move_down",
    9 : "zoom_in",
    0 : "zoom_out"
}

# Listen for incoming connections
server_socket.listen(1)
print(f"Server listening on {host}:{port}")
my_preset , my_theme = 0, 0

while True:
    # Wait for a connection
    client_socket, client_address = server_socket.accept()
    try:
        print(f"Connection from {client_address}")

        # Receive the data in small chunks and retransmit it
        loading = False
        start_time = None
        preset = client_socket.recv(1024).decode()
        if preset[0] == "P":
            my_preset = preset[1:]
        theme = client_socket.recv(1024).decode()
        if theme[0] == "T":
            if theme[1:] != my_theme:
                print("changed to ", theme[1:])
                my_theme = theme[1:]
                colour = int(my_theme)
                update_reg_map(colour, dec_to_hex(zoom), dec_to_hex(Im), dec_to_hex(Re), dim, it)
                time.sleep(2)
                frame = imgen_vdma.readframe() 
        julia = client_socket.recv(1024).decode()
        if julia[0] == "J":
            is_julia = julia[1:]
            if is_julia == "1":
                print("julia")
                my_theme += 16
        print(preset, theme, julia)
        connection = True
        while connection:
            data = client_socket.recv(1024)
            send_message = "ACK"
            client_socket.sendall(send_message.encode())
            print("sent ACK")
            if data:
                for i in range(10):
                    if int(data.decode('utf-8')) == i:
                        message = mapping[i]
                #message = data.decode('utf-8')
                print(f"Received message:{message}")
                #respond
                if message == "move_left_down":
                    cropped_frame, x, y, out = move_left(frame, x, y)
                    cropped_frame, x, y, out = move_down(frame, x, y)
                if message == "move_left_up":
                    cropped_frame, x, y, out = move_left(frame, x, y)
                    cropped_frame, x, y, out = move_up(frame, x, y)
                if message == "move_right_down":
                    cropped_frame, x, y, out = move_right(frame, x, y)
                    cropped_frame, x, y, out = move_down(frame, x, y)
                if message == "move_right_up":
                    cropped_frame, x, y, out = move_right(frame, x, y)
                    cropped_frame, x, y, out = move_up(frame, x, y)
                if message == "move_left":
                    cropped_frame, x, y, out = move_left(frame, x, y)
                if message == "move_right":
                    cropped_frame, x, y, out = move_right(frame, x, y)
                if message == "move_up":
                    cropped_frame, x, y, out = move_up(frame, x, y)
                if message == "move_down":
                    cropped_frame, x, y, out = move_down(frame, x, y)
                if message == "zoom_in":
                    cropped_frame, image_w, image_h, out= zoom_in(frame, x, y, image_w, image_h)
                if message == "zoom_out":
                    cropped_frame, image_w, image_h, out= zoom_out(frame, x, y, image_w, image_h)
                if message == "reset":
                    reset()
                    time.sleep(loading_time)
                    frame = imgen_vdma.readframe()
                if message == "close":
                    print("Connection closed")
                    client_socket.close()
                    connection = False
                if out and not loading:
                    print("out")
                    new_Re, new_Im, zoom, image_Re, image_Im = generate_new_frame(zoom, Re, Im, image_w, image_h)
                    og_x, og_y = x, y
                    og_w, og_h = image_w, image_h
                    print("Image", image_Re, image_Im)
                    print("Frame", new_Re, new_Im)
                    #new_x, new_y = get_new_xy(x, y, Re, new_Re, Im, new_Im, image_Re, image_Im)
                    #print(new_x, new_y)
                    Re = new_Re
                    Im = new_Im

                    loading = True
                    start_time = time.time()
                    x_buffer = 0
                    y_buffer = 0

                elif loading:
                    if time.time() - start_time >= loading_time:
                        bg_frame = imgen_vdma.readframe()
                        frame = bg_frame
                        image_w = 640 + (image_w - og_w)
                        image_h = 480 + (image_h - og_h)
                        loading = False
                        x, y = center_x + (x - og_x), center_y + (y - og_y)
                        print("image loaded")
                    else:
                        print("buffer")
                hdmi_out.writeframe(cropped_frame)
            else:
                break
    finally:
        # Clean up the connection
        client_socket.close()

Server listening on 0.0.0.0:12337
Connection from ('192.168.2.4', 6290)
changed to  0
0
P1 T0
Received message:move_left
Received message:move_leftmove_left
Received message:move_left
Received message:move_leftmove_leftmove_left
Received message:move_left
Received message:move_leftmove_left
Received message:move_left
Received message:move_leftmove_left_upmove_left_upmove_left_up
Received message:move_left_up
Received message:move_left
Received message:move_right
Received message:move_rightmove_right_downmove_right_down
Received message:move_right_down
Received message:move_right_downmove_right_downmove_right_downmove_right_downmove_right_down
Received message:move_right_down
Received message:move_right_downmove_right_downmove_right_downmove_right_down
Received message:zoom_out
Received message:zoom_outzoom_outzoom_out
Received message:zoom_out
Received message:zoom_outzoom_outzoom_out
Received message:zoom_outzoom_out
Received message:zoom_out
Received message:zoom_outzoom_outzoom_out


Received message:move_up
buffer
Received message:move_upmove_upmove_up
buffer
Received message:move_up
buffer
Received message:move_upmove_up
buffer
Received message:move_up
buffer
Received message:move_up
buffer
Received message:close
Connection closed
image loaded
Connection from ('192.168.2.4', 6293)
changed to  1
1
P1 T1
Received message:move_right
Received message:move_right_downmove_right_down
Received message:move_right_downmove_right
Received message:move_right
Received message:move_rightmove_right
Received message:move_right
Received message:move_rightmove_rightmove_right
Received message:move_right
Received message:move_rightmove_rightmove_right
Received message:move_right_down
Received message:move_right_downmove_right_downmove_right_down
Received message:move_right_downmove_right_down
Received message:move_right_downmove_right_down
Received message:move_down
Received message:move_down
Received message:move_downmove_left_downmove_left_downmove_left_downmove_left_down
Receive

buffer
Received message:move_left_downmove_left_downmove_left_down
buffer
Received message:move_left_downmove_left_downmove_left_down
buffer
Received message:move_left_down
buffer
Received message:move_left_downmove_downmove_down
buffer
Received message:move_downmove_downmove_down
buffer
Received message:move_down
buffer
Received message:move_down
buffer
Received message:move_downmove_downmove_downmove_downmove_down
buffer
Received message:move_down
buffer
Received message:move_down
buffer
Received message:move_downmove_downmove_down
buffer
Received message:move_downmove_downmove_down
buffer
Received message:move_down
buffer
Received message:move_down
buffer
Received message:move_downmove_downmove_downmove_downmove_down
buffer
Received message:move_down
buffer
Received message:move_down
buffer
Received message:move_right_downmove_right_downmove_right_downmove_right_down
buffer
Received message:move_right_down
image loaded
Received message:move_right_downmove_right_downmove_right_down
R

buffer
Received message:move_right_downmove_right_downmove_right_downmove_right_down
buffer
Received message:move_right_downmove_right_down
buffer
Received message:move_down
image loaded
Received message:move_downmove_downmove_downmove_down
Received message:move_left_down
Received message:move_left_downmove_leftmove_left
Received message:move_leftmove_left
Received message:move_left
Received message:move_leftmove_leftmove_left
Received message:move_left
Received message:move_left
Received message:move_leftmove_leftmove_leftzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_in
Received message:zoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzo

buffer
Received message:zoom_inzoom_inzoom_in
buffer
Received message:zoom_inzoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_inzoom_inzoom_inzoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_inzoom_inzoom_inzoom_inzoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_inzoom_inzoom_inzoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_inzoom_inzoom_inzoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_inzoom_inzoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_inzoom_inzoom_inzoom_inzoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_inzoom_inzoom_in
buffer
Received message:zoom_in
image loaded
Received message:zoom_inzoom_inzoom_in
out
p 0.234375 0.234375
Small_d 1.2679132300768932 0.9509349225576699
centre 0.2622529483825863 -0.023776238397804184
big_distance 2.028661168123029 1.5214958760922717
5
Im

...

Received message:move_downmove_downmove_downmove_downmove_right_downmove_right_down
Received message:move_right_down
Received message:move_right_downmove_right_downmove_right_down
Received message:move_right_downmove_right_down
Received message:move_right_down
Received message:move_right_downmove_right_downmove_right_downmove_right_down
Received message:move_right_downmove_right_down
Received message:move_right_down
Received message:move_right_downmove_right_downmove_right_downmove_down
Received message:zoom_inzoom_in
Received message:move_down
Received message:move_down
Received message:move_downmove_downmove_downmove_downmove_down
Received message:move_down
Received message:move_down
Received message:move_downmove_downmove_downmove_down
Received message:move_down
Received message:move_downmove_downmove_downmove_down
Received message:move_down
Received message:move_down
Received message:move_downmove_down
Received message:move_downmove_downmove_down
Received message:move_down
Received

Received message:zoom_in
image loaded
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_in
Received message:zoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_in
Received message:zoom_inzoom_in
Received message:zoom_in
out
p 0.2548828125 0.24348958333333334
Small_d 1.2335584015672012 0.9251688011754009
centre 0.3218982227219612 -0.003724262603355122
big_distance 1.9736934425075219 1.4802700818806414
1
Image -0.2948809780616394 0.45886013798434533
Frame -0.6649484985317997 0.7364107783369656
Rec

Received message:zoom_in
Received message:zoom_inzoom_inzoom_in
Received message:zoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_inzoom_in
Received message:zoom_in
Received message:zoom_inzoom_inzoom_in
Received message:zoom_in
out
p 0.2646484375 0.2825520833333333
Small_d 0.8388197130656968 0.6291147847992726
centre 0.4004411990717478 0.32683084031660603
big_distance 1.342111540905115 1.0065836556788361
1
Image -0.018968657461100602 0.6413882327162423
Frame -0.2706145713808097 0.8301226681560241
Received message:zoom_inzoom_inzoom_inzoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_inzoom_inzoom_inzoom_in
buffer
Received message:zoom_in
buffer
Received message:zoom_inzoom_inzoom_in
buffer
Received 

buffer
Received message:zoom_outzoom_outzoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received message:zoom_outzoom_outzoom_outzoom_out
buffer
Received message:zoom_outzoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received message:zoom_outzoom_outzoom_out
buffer
Received message:zoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received message:zoom_outzoom_outzoom_out
buffer
Received message:zoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received message:zoom_outzoom_outzoom_outzoom_out
buffer
Received message:zoom_outzoom_out
buffer
Received message:zoom_out
image loaded
Received message:zoom_outzoom_outzoom_out
out
p 0.15625 0.15625
Small_d 0.824981809111716 0.618736356833787
centre 0.3712663836696849 0.31736135214957545
big_distance 1.3199708945787456 0.9899781709340592
1
Image -0.041224520886173094 0.626729530566469
Frame -0.2887190636196879 0.8123504376166051
Received message:zoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received 

buffer
Received message:zoom_outzoom_outzoom_out
buffer
Received message:zoom_outzoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received message:zoom_outzoom_outzoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received message:zoom_outzoom_outzoom_outzoom_out
buffer
Received message:zoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received message:zoom_outzoom_outzoom_out
buffer
Received message:zoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received message:zoom_outzoom_outzoom_outzoom_out
buffer
Received message:zoom_out
buffer
Received message:zoom_outzoom_outzoom_outzoom_out
buffer
Received message:zoom_outzoom_out
buffer
Received message:zoom_out
image loaded
Received message:zoom_outzoom_outzoom_out
Received message:zoom_outzoom_out
Received message:zoom_out
Received message:zoom_outzoom_outzoom_outzoom_out
Received message:zoom_out
Received message:zoom_outzoom_outzoom_out
Received message:zoom_outzoom_outzoom_out
Received message:zoom_out
Rec